Multi transport mode choice modelling.  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
import pylogit as pl

In [3]:
from collections import OrderedDict

In [4]:
data = pd.read_csv("transportation_data.csv")

In [5]:
data.head()

,TRAVELER,MODE,TTME,INVC,INVT,HINC
0,1,0,69,59,100,35
1,1,0,34,31,372,35
2,1,0,35,25,417,35
3,1,1,0,10,180,35
4,2,0,64,58,68,30


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 840 entries, 0 to 839
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   TRAVELER  840 non-null    int64
 1   MODE      840 non-null    int64
 2   TTME      840 non-null    int64
 3   INVC      840 non-null    int64
 4   INVT      840 non-null    int64
 5   HINC      840 non-null    int64
dtypes: int64(6)
memory usage: 39.5 KB


Each Traveller is given a choice of 4 modes of transportation.  
1- Plane , 2- Train , 3 - Bus , 4 - Car.  

In [7]:
mode_id = (1,2,3,4)
repeat = len(data['MODE'])/4
mode_id = mode_id*int(repeat)

In [8]:
data['MODE_ID'] = mode_id

In [9]:
data.head(12)

,TRAVELER,MODE,TTME,INVC,INVT,HINC,MODE_ID
0,1,0,69,59,100,35,1
1,1,0,34,31,372,35,2
2,1,0,35,25,417,35,3
3,1,1,0,10,180,35,4
4,2,0,64,58,68,30,1
5,2,0,44,31,354,30,2
6,2,0,53,25,399,30,3
7,2,1,0,11,255,30,4
8,3,0,69,115,125,40,1
9,3,0,34,98,892,40,2


In [10]:
data1 = data.copy()
variable = ['TTME', 'INVC', 'INVT']
basic_specification = OrderedDict()
basic_names = OrderedDict()


In [11]:
basic_specification['intercept'] = [2,3,4]
basic_names['intercept'] = ['train:intercept', 'bus:intercept', 'car:intercept']
##
for col in variable:
    basic_specification[col] = [[1,2,3,4]]
    basic_names[col] = [col]
    

In [12]:
basic_names

OrderedDict([('intercept',
              ['train:intercept', 'bus:intercept', 'car:intercept']),
             ('TTME', ['TTME']),
             ('INVC', ['INVC']),
             ('INVT', ['INVT'])])

In [13]:
basic_specification

OrderedDict([('intercept', [2, 3, 4]),
             ('TTME', [[1, 2, 3, 4]]),
             ('INVC', [[1, 2, 3, 4]]),
             ('INVT', [[1, 2, 3, 4]])])

In [14]:
model1 = pl.create_choice_model(data = data1,
                               alt_id_col = 'MODE_ID',
                               obs_id_col = 'TRAVELER',
                               choice_col = 'MODE',
                               specification = basic_specification,
                               model_type = 'MNL',
                               names = basic_names)

In [15]:
model1.fit_mle(np.zeros(6))
model1.get_statsmodels_summary()

Log-likelihood at zero: -291.1218
Initial Log-likelihood: -291.1218
Estimation Time for Point Estimation: 0.04 seconds.
Final log-likelihood: -192.8885


/Users/anushabanninthaya/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/_minimize.py:522: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                        MODE   No. Observations:                  210
Model:             Multinomial Logit Model   Df Residuals:                      204
Method:                                MLE   Df Model:                            6
Date:                     Fri, 01 Jan 2021   Pseudo R-squ.:                   0.337
Time:                             11:11:59   Pseudo R-bar-squ.:               0.317
AIC:                               397.777   Log-Likelihood:               -192.889
BIC:                               417.860   LL-Null:                      -291.122
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
train:intercept    -0.7867      0.603     -1.305      0.192      -1.968       0.394
bus:intercept      -1.4336      0.681     -2.106      0.035      -2.768      -0.099
car:intercept      -4.7399      0.868     -5.464      0.000      -6.440      -3.040
TTME               -0.0969      0.010     -9.368      0.000      -0.117      -0.077
INVC               -0.0139      0.007     -2.092      0.036      -0.027      -0.001
INVT               -0.0040      0.001     -4.704      0.000      -0.006      -0.002
===================================================================================
"""

The P value indicates that the variables 'intercept for Car' , TTME , INVT are statistically significant.

In [16]:
data1.head()

,TRAVELER,MODE,TTME,INVC,INVT,HINC,MODE_ID,intercept
0,1,0,69,59,100,35,1,1.0
1,1,0,34,31,372,35,2,1.0
2,1,0,35,25,417,35,3,1.0
3,1,1,0,10,180,35,4,1.0
4,2,0,64,58,68,30,1,1.0


In [17]:
data1['prob'] = model1.predict(data1)

In [18]:
data1.head(5)

,TRAVELER,MODE,TTME,INVC,INVT,HINC,MODE_ID,intercept,prob
0,1,0,69,59,100,35,1,1.0,0.048330
1,1,0,34,31,372,35,2,1.0,0.325514
2,1,0,35,25,417,35,3,1.0,0.140507
3,1,1,0,10,180,35,4,1.0,0.485649
4,2,0,64,58,68,30,1,1.0,0.149547


In [19]:
## get the index of the maximum probability for each Traveler 
idx = data1.groupby(['TRAVELER'])['prob'].idxmax()
data1['pred_selection'] = 0
## set the selection as 1 for the indexes.
data1.loc[idx, 'pred_selection'] = 1

In [20]:
data1.head()

,TRAVELER,MODE,TTME,INVC,INVT,HINC,MODE_ID,intercept,prob,pred_selection
0,1,0,69,59,100,35,1,1.0,0.048330,0
1,1,0,34,31,372,35,2,1.0,0.325514,0
2,1,0,35,25,417,35,3,1.0,0.140507,0
3,1,1,0,10,180,35,4,1.0,0.485649,1
4,2,0,64,58,68,30,1,1.0,0.149547,0


In [21]:
## How much is the predicted share for each mode of transport ? 
def label_mode(row, pred_or_actual):
    if pred_or_actual == 'pred':
        
        if row['pred_selection'] == 1:
            return row['MODE_ID']
        else:
            return 0
    elif pred_or_actual == 'actual':
        if row['MODE'] == 1:
            return row['MODE_ID']
        else:
            return 0
        
    
data1['pred_mode'] = data1[['MODE_ID','pred_selection']].apply(lambda row: label_mode(row, 'pred'), axis = 1)
data1['actual_mode'] = data1[['MODE_ID','MODE']].apply(lambda row: label_mode(row, 'actual'), axis = 1)
##
print("Predicted share of each mode of transport:")
data1[data1['pred_mode'] !=0]['pred_mode'].value_counts(normalize = True)

Predicted share of each mode of transport:


4    0.319048
2    0.304762
1    0.261905
3    0.114286
Name: pred_mode, dtype: float64

In [33]:
crosstab_data = data1[['pred_mode','actual_mode','TRAVELER']].groupby(['TRAVELER'])['pred_mode','actual_mode'].agg({'max'}).reset_index()

<ipython-input-33-6c669203bc80>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  crosstab_data = data1[['pred_mode','actual_mode','TRAVELER']].groupby(['TRAVELER'])['pred_mode','actual_mode'].agg({'max'}).reset_index()


In [44]:
crosstab_data.columns= ['TRAVELER', 'pred_mode', 'actual_mode']
crosstab_data.head()
print(crosstab_data.shape)


(210, 3)


In [43]:
pd.crosstab(crosstab_data['actual_mode'], crosstab_data['pred_mode'])

pred_mode,1,2,3,4
actual_mode,,,,
1,39,4,0,15
2,6,49,1,7
3,3,3,23,1
4,7,8,0,44


In [45]:
from sklearn.metrics import classification_report 
print(classification_report(crosstab_data['actual_mode'], crosstab_data['pred_mode']))


              precision    recall  f1-score   support

           1       0.71      0.67      0.69        58
           2       0.77      0.78      0.77        63
           3       0.96      0.77      0.85        30
           4       0.66      0.75      0.70        59

    accuracy                           0.74       210
   macro avg       0.77      0.74      0.75       210
weighted avg       0.75      0.74      0.74       210



The model has a good prediction rate with the least mis-classification for Bus - mode 3, followed by that for mode 2 -Train.    

In [46]:
## Adding the effect of Income on travel mode choice 

data2 = data.copy()
basic_specification['HINC'] = [2,3,4]
basic_names['HINC'] = ['train:income', 'bus:income', 'car:income']

In [47]:
basic_specification

OrderedDict([('intercept', [2, 3, 4]),
             ('TTME', [[1, 2, 3, 4]]),
             ('INVC', [[1, 2, 3, 4]]),
             ('INVT', [[1, 2, 3, 4]]),
             ('HINC', [2, 3, 4])])

In [48]:
basic_names

OrderedDict([('intercept',
              ['train:intercept', 'bus:intercept', 'car:intercept']),
             ('TTME', ['TTME']),
             ('INVC', ['INVC']),
             ('INVT', ['INVT']),
             ('HINC', ['train:income', 'bus:income', 'car:income'])])

In [49]:
model2 = pl.create_choice_model(data = data2,
                               alt_id_col = 'MODE_ID',
                               obs_id_col = 'TRAVELER',
                               choice_col = 'MODE',
                               specification = basic_specification,
                               model_type = 'MNL',
                               names = basic_names)
model2.fit_mle(np.zeros(9))
model2.get_statsmodels_summary()

Log-likelihood at zero: -291.1218
Initial Log-likelihood: -291.1218
Estimation Time for Point Estimation: 0.09 seconds.
Final log-likelihood: -182.2186


/Users/anushabanninthaya/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/_minimize.py:522: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                        MODE   No. Observations:                  210
Model:             Multinomial Logit Model   Df Residuals:                      201
Method:                                MLE   Df Model:                            9
Date:                     Fri, 01 Jan 2021   Pseudo R-squ.:                   0.374
Time:                             14:57:40   Pseudo R-bar-squ.:               0.343
AIC:                               382.437   Log-Likelihood:               -182.219
BIC:                               412.561   LL-Null:                      -291.122
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
train:intercept     1.2421      0.817      1.521      0.128      -0.359       2.843
bus:intercept      -0.1844      0.897     -0.206      0.837      -1.942       1.573
car:intercept      -4.2474      1.007     -4.220      0.000      -6.220      -2.275
TTME               -0.0953      0.010     -9.202      0.000      -0.116      -0.075
INVC               -0.0045      0.007     -0.624      0.533      -0.019       0.010
INVT               -0.0037      0.001     -4.222      0.000      -0.005      -0.002
train:income       -0.0559      0.015     -3.640      0.000      -0.086      -0.026
bus:income         -0.0231      0.016     -1.404      0.160      -0.055       0.009
car:income          0.0021      0.012      0.174      0.862      -0.022       0.026
===================================================================================
"""

Model's BIC has reduced a bit , indicating a slightly better model.
The p value indicates that the variables 'intercept for Car' , TTME , INVT and interaction of Income on Train mode are statistically significant.   

In [50]:
data2['prob'] = model2.predict(data2)

In [58]:
data2.head(5)
## get the index of the maximum probability for each Traveler 
idx2 = data2.groupby(['TRAVELER'])['prob'].idxmax()
data2['pred_selection'] = 0
## set the selection as 1 for the indexes.
data2.loc[idx2, 'pred_selection'] = 1
##
print(data2.head())
data2['pred_mode'] = data2[['MODE_ID','pred_selection']].apply(lambda row: label_mode(row, 'pred'), axis = 1)
data2['actual_mode'] = data2[['MODE_ID','MODE']].apply(lambda row: label_mode(row, 'actual'), axis = 1)
##
print("\n Predicted share of each mode of transport:")
print(data2[data2['pred_mode'] !=0]['pred_mode'].value_counts(normalize = True))
##
crosstab_data2 = data2[['pred_mode','actual_mode','TRAVELER']].groupby(['TRAVELER'])['pred_mode','actual_mode'].agg({'max'}).reset_index()
##
crosstab_data2.columns = ['TRAVELER', 'pred_mode', 'actual_mode']
crosstab_data2.head()
print(crosstab_data2.shape)


   TRAVELER  MODE  TTME  INVC  INVT  HINC  MODE_ID  intercept      prob  \
0         1     0    69    59   100    35        1        1.0  0.048879   
1         1     0    34    31   372    35        2        1.0  0.281245   
2         1     0    35    25   417    35        3        1.0  0.168508   
3         1     1     0    10   180    35        4        1.0  0.501368   
4         2     0    64    58    68    30        1        1.0  0.136002   

   pred_selection  pred_mode  actual_mode  
0               0          0            0  
1               0          0            0  
2               0          0            0  
3               1          4            4  
4               0          0            0  

 Predicted share of each mode of transport:
2    0.319048
4    0.309524
1    0.257143
3    0.114286
Name: pred_mode, dtype: float64
(210, 3)


<ipython-input-58-54b40b2fa7cb>:15: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  crosstab_data2 = data2[['pred_mode','actual_mode','TRAVELER']].groupby(['TRAVELER'])['pred_mode','actual_mode'].agg({'max'}).reset_index()


In [56]:
##
pd.crosstab(crosstab_data2['actual_mode'], crosstab_data2['pred_mode'])

pred_mode,1,2,3,4
actual_mode,,,,
1,40,5,0,13
2,6,50,1,6
3,1,3,23,3
4,7,9,0,43


In [57]:
print(classification_report(crosstab_data2['actual_mode'], crosstab_data2['pred_mode']))

              precision    recall  f1-score   support

           1       0.74      0.69      0.71        58
           2       0.75      0.79      0.77        63
           3       0.96      0.77      0.85        30
           4       0.66      0.73      0.69        59

    accuracy                           0.74       210
   macro avg       0.78      0.74      0.76       210
weighted avg       0.75      0.74      0.74       210



For mode 1 and 2 i.e plane and train , the true positive has increased by one.    

In [70]:
l1 = model1.fit_summary['Fitted Log-Likelihood']
l2 = model2.fit_summary['Fitted Log-Likelihood']


In [71]:
## chisq test 
from scipy.stats.distributions import chi2
def lrtest(lmax, lmin , dof):
    llr = 2*(lmax - lmin)
    p = chi2.sf(llr, dof)
    return p

lrtest(l2,l1,3)

8.948392116314905e-05

Scenario modelling :  

If the Train mode reduces its travel time by 10%, what is the effect on the market share of each mode of transport ?  

In [59]:
data2_new = data.copy()
data2_new['INVT'] = data2_new.apply(lambda row : row['INVT']*0.9 if row['MODE_ID'] ==2 else row['INVT'] , axis = 1)


In [60]:
## predict the probability of selection of each choice 
data2_new['prob'] = model2.predict(data2_new)
## get the index of the maximum probability for each Traveler 
idx2 = data2_new.groupby(['TRAVELER'])['prob'].idxmax()
data2_new['pred_selection'] = 0
## set the selection as 1 for the indexes.
data2_new.loc[idx2, 'pred_selection'] = 1
## set the values of actual_mode and predicted mode columns
data2_new['pred_mode'] = data2_new[['MODE_ID','pred_selection']].apply(lambda row: label_mode(row, 'pred'), axis = 1)
data2_new['actual_mode'] = data2_new[['MODE_ID','MODE']].apply(lambda row: label_mode(row, 'actual'), axis = 1)
## market share of each mode of travel 
print("Predicted share of each mode of transport with 10% reduction in train travel time:")
print(data2_new[data2_new['pred_mode'] !=0]['pred_mode'].value_counts(normalize = True))
pred_share_new = data2_new[data2_new['pred_mode'] !=0]['pred_mode'].value_counts(normalize = True)
pred_share_old = data2[data2['pred_mode'] !=0]['pred_mode'].value_counts(normalize = True)

Predicted share of each mode of transport with 10% reduction in train travel time:
2    0.338095
4    0.300000
1    0.252381
3    0.109524
Name: pred_mode, dtype: float64


In [61]:
pred_share_old

2    0.319048
4    0.309524
1    0.257143
3    0.114286
Name: pred_mode, dtype: float64

In [62]:
## Change in market share of each mode 
(pred_share_new - pred_share_old)/pred_share_old

2    0.059701
4   -0.030769
1   -0.018519
3   -0.041667
Name: pred_mode, dtype: float64

Its seen that the with 10% reduction in In-vehicle-Train time , the predicted market share of Train mode increases by 5%. Mode of Bus is impacted by this change with a maximum reduction of market share of 4%.

Impact of Adding Interaction Terms:

In [63]:
## Adding interaction terms 
data3 = data.copy()
data3['TTME:income'] = data3['TTME']*data3['HINC']
data3['INVC:income'] = data3['INVC']*data3['HINC']
data3['INVT:income'] = data3['INVT']*data3['HINC']


In [64]:
interaction = data3.iloc[:,-3:].columns
for col in interaction:
    basic_specification[col] = [[1,2,3,4]]
    basic_names[col] = [col]

In [65]:
basic_specification

OrderedDict([('intercept', [2, 3, 4]),
             ('TTME', [[1, 2, 3, 4]]),
             ('INVC', [[1, 2, 3, 4]]),
             ('INVT', [[1, 2, 3, 4]]),
             ('HINC', [2, 3, 4]),
             ('TTME:income', [[1, 2, 3, 4]]),
             ('INVC:income', [[1, 2, 3, 4]]),
             ('INVT:income', [[1, 2, 3, 4]])])

In [66]:
basic_names

OrderedDict([('intercept',
              ['train:intercept', 'bus:intercept', 'car:intercept']),
             ('TTME', ['TTME']),
             ('INVC', ['INVC']),
             ('INVT', ['INVT']),
             ('HINC', ['train:income', 'bus:income', 'car:income']),
             ('TTME:income', ['TTME:income']),
             ('INVC:income', ['INVC:income']),
             ('INVT:income', ['INVT:income'])])

In [67]:
model3 = pl.create_choice_model(data = data3,
                               alt_id_col = 'MODE_ID',
                               obs_id_col = 'TRAVELER',
                               choice_col = 'MODE',
                               specification = basic_specification,
                               model_type = 'MNL',
                               names = basic_names)
model3.fit_mle(np.zeros(12))
model3.get_statsmodels_summary()

Log-likelihood at zero: -291.1218
Initial Log-likelihood: -291.1218
Estimation Time for Point Estimation: 0.09 seconds.
Final log-likelihood: -180.1039


/Users/anushabanninthaya/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/_minimize.py:522: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                        MODE   No. Observations:                  210
Model:             Multinomial Logit Model   Df Residuals:                      198
Method:                                MLE   Df Model:                           12
Date:                     Fri, 01 Jan 2021   Pseudo R-squ.:                   0.381
Time:                             18:43:16   Pseudo R-bar-squ.:               0.340
AIC:                               384.208   Log-Likelihood:               -180.104
BIC:                               424.373   LL-Null:                      -291.122
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
train:intercept     2.7964      1.374      2.036      0.042       0.104       5.489
bus:intercept       1.3521      1.469      0.920      0.357      -1.527       4.231
car:intercept      -2.2819      1.790     -1.274      0.202      -5.791       1.227
TTME               -0.0768      0.020     -3.883      0.000      -0.116      -0.038
INVC               -0.0121      0.016     -0.739      0.460      -0.044       0.020
INVT               -0.0065      0.002     -3.429      0.001      -0.010      -0.003
train:income       -0.0996      0.033     -3.041      0.002      -0.164      -0.035
bus:income         -0.0650      0.035     -1.873      0.061      -0.133       0.003
car:income         -0.0546      0.046     -1.193      0.233      -0.144       0.035
TTME:income        -0.0006      0.001     -1.079      0.281      -0.002       0.001
INVC:income         0.0002      0.000      0.576      0.564      -0.001       0.001
INVT:income      7.517e-05   4.23e-05      1.776      0.076    -7.8e-06       0.000
===================================================================================
"""

By adding the interaction terms, only TTME , INVT and train:income are statistically significant , indicating that the new features are not improving the model.   

In [68]:
l3 = model3.fit_summary['Fitted Log-Likelihood']

In [72]:
lrtest(l3,l2,3)

0.23772207535235912

Its seen that the model with interaction terms of income with the mode features is not a better model.   